# Chapter 3 - Exercise 2
#### Author: *John Benedick Estrada*
---
**Exercise:** My implementation of `reachable_nodes_bfs` is efficient in the sense that it is in $O(n + m)$, but it incurs a lot of overhead adding nodes to the queue and removing them.  NetworkX provides a simple, fast implementation of BFS, available from [the NetworkX repository on GitHub](https://github.com/networkx/networkx/blob/master/networkx/algorithms/components/connected.py).

Here is a version I modified to return a set of nodes:
```
def plain_bfs(G, start):
    """A fast BFS node generator"""
    seen = set()
    nextlevel = {start}
    while nextlevel:
        thislevel = nextlevel
        nextlevel = set()
        for v in thislevel:
            if v not in seen:
                seen.add(v)
                nextlevel.update(G[v])
    return seen
```

Compare this function to `reachable_nodes_bfs` and see which is faster.  Then see if you can modify this function to implement a faster version of `shortest_path_dijkstra`

In [1]:
import networkx as nx
from collections import deque

##### Implementation of `make_ring_lattice` directly lifted from the book

In [2]:
def adjacent_edges(nodes, halfk):
    """Yields edges between each node and `halfk` neighbors.
    
    halfk: number of edges from each node
    """
    n = len(nodes)
    for i, u in enumerate(nodes):
        for j in range(i+1, i+halfk+1):
            v = nodes[j % n]
            yield u, v


def make_ring_lattice(n, k):
    """Makes a ring lattice with `n` nodes and degree `k`.
    
    Note: this only works correctly if k is even.
    
    n: number of nodes
    k: degree of each node
    """
    G = nx.Graph()
    nodes = range(n)
    G.add_nodes_from(nodes)
    G.add_edges_from(adjacent_edges(nodes, k//2))
    return G

In [3]:
# Implementation from the book.
def reachable_nodes_bfs(G, start):
    """Finds reachable nodes by BFS.
    
    G: graph
    start: node to start at
    
    returns: set of reachable nodes
    """
    seen = set()
    queue = deque([start])
    while queue:
        node = queue.popleft()
        if node not in seen:
            seen.add(node)
            queue.extend(G.neighbors(node))
    return seen


# Optimized version of `reachable_node_bfs`
def plain_bfs(G, start):
    """A fast BFS node generator"""
    seen = set()
    nextlevel = {start}
    while nextlevel:
        thislevel = nextlevel
        nextlevel = set()
        for v in thislevel:
            if v not in seen:
                seen.add(v)
                nextlevel.update(G[v])
    return seen

##### Simple benchmark for the two implementation of Breadth-first search

In [4]:
lattice = make_ring_lattice(100, 4)

In [5]:
%timeit plain_bfs(lattice, 0)
%timeit reachable_nodes_bfs(lattice, 0)

120 µs ± 5.25 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
67.8 µs ± 6.63 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
# Implementation of Dijkstra's SPA from the book:
def shortest_path_dijkstra(G, source):
    """Finds shortest paths from `source` to all other nodes.
    
    G: graph
    source: node to start at
    
    returns: make from node to path length
    """
    dist = {source: 0}
    queue = deque([source])
    while queue:
        node = queue.popleft()
        new_dist = dist[node] + 1

        neighbors = set(G[node]).difference(dist)
        for n in neighbors:
            dist[n] = new_dist
        
        queue.extend(neighbors)
    return dist


# My implementation of Dijkstra's SPA:
def shortest_path_dijkstra_2(G, source):
    dist = {source: 0}
    next_unvisited = {source}
    while next_unvisited:
        unvisited = next_unvisited
        next_unvisited = set()

        for node in unvisited:
            new_dist = dist[node] + 1

            neighbors = set(G[node]).difference(dist)
            for n in neighbors:
                dist[n] = new_dist
                next_unvisited.add(n)

    return dist

##### Testing implementations of Dijkstra's shortest path algorithm (SPA)

In [7]:
# Dijkstra's shortest path algorithm (SPA). 
# NetworkX implementation:
spa_nx = nx.dijkstra_predecessor_and_distance(lattice, 0)[1]
# The book's implementation:
spa_1 = shortest_path_dijkstra(lattice, 0)
# My implementation:
spa_2 = shortest_path_dijkstra_2(lattice, 0)

assert spa_nx == spa_1
assert spa_nx == spa_2
assert spa_1 == spa_2
print("Success!")

Success!


##### Simple benchmark for the implementations of Dijkstra's SPA

In [8]:
print("NetworkX's implementation: ", end="")
%timeit nx.dijkstra_predecessor_and_distance(lattice, 0)[1]
print("The book's implementation: ", end="")
%timeit shortest_path_dijkstra(lattice, 0)
print("My implementation:         ", end="")
%timeit shortest_path_dijkstra_2(lattice, 0)

NetworkX's implementation: 237 µs ± 14.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
The book's implementation: 150 µs ± 4.45 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
My implementation:         140 µs ± 2.35 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
